In [1]:
!pip install midiSynth
from midiSynth.synth import MidiSynth
midi_synth = MidiSynth()

In [2]:
import os
import pickle
import torch

In [3]:
from utils import write_midi
from models import TransformerModel, network_paras

## 1. Prepare dictionary

In [4]:
!gdown --id 17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP
!unzip co-representation.zip -d ../../dataset/
!rm co-representation.zip

In [5]:
path_dictionary = '../../dataset/co-representation/dictionary.pkl'
assert os.path.exists(path_dictionary)

In [6]:
dictionary = pickle.load(open(path_dictionary, 'rb'))
event2word, word2event = dictionary

In [7]:
# config
n_class = []   # num of classes for each token
for key in event2word.keys():
    n_class.append(len(dictionary[0][key]))
n_token = len(n_class)

## 2. Prepare model

In [8]:
!gdown --id 19Seq18b2JNzOamEQMG1uarKjj27HJkHu --output exp/pretrained_transformer.zip
!unzip exp/pretrained_transformer.zip -d exp/
!rm exp/pretrained_transformer.zip
os.listdir('exp/pretrained_transformer')

In [9]:
path_saved_ckpt = 'exp/pretrained_transformer/loss_25_params.pt'
assert os.path.exists(path_saved_ckpt)

In [10]:
# init model
net = TransformerModel(n_class, is_training=False)
net.cuda()
net.eval()

net.load_state_dict(torch.load(path_saved_ckpt))

>>>>>: [56, 135, 18, 4, 87, 18, 42, 5]
 [o] using RNN backend.


<All keys matched successfully>

## 3. Start generating

### 3.1 setup parameters

In [15]:
emotion_tag = 4  # the target emotion class you want. It should belongs to [1,2,3,4].
path_outfile = 'test' # output midi file name

In [16]:
res, _ = net.inference_from_scratch(dictionary, emotion_tag, n_token=8, display=False)
write_midi(res, path_outfile + '.mid', word2event)

midi_synth.play_midi(path_outfile + '.mid')
midi_synth.midi2audio(path_outfile + '.mid', path_outfile + '.mp3')

------ initiate ------
0               | 0               | 0               | Emotion         | 0               | 0               | 0               | 4               | 
0               | 0               | Bar             | Metrical        | 0               | 0               | 0               | 0               | 
------ generate ------

--------[Done]--------
(253, 8)
